In [1]:
import json
import os
import re
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml

from penalty_calc import calculate_total_cost_including_student_conflicts
from random_student_sectioning import RandomStudentSectioning
from solution_to_xml import generate_xml, output_itc2007_cb, output_itc2007_post


In [2]:

show_graphs = False

output_folder_path = f'output\\.analysis\\{time.strftime("%Y%m%d-%H%M%S")}'
os.makedirs(output_folder_path)

assets_folder_path = os.path.join(output_folder_path, 'assets')
os.makedirs(assets_folder_path)

itc2007_track2_path = 'D:\\Desktop\\Datasets\\post'
itc2007_track3_path = 'D:\\Desktop\\Datasets\\curriculum'
itc2019_path = 'D:\\Desktop\\Datasets\\2019'

In [3]:
from parse_input import parse_xml, parse_itc2007_post_enrolment, parse_itc2007_curriculum_based


def get_instance_by_dataset_and_instance_file_name(dataset, instance_file_name):
    if dataset == 'itc2007_track2':
        return next((file for file in itc2007_track2_files if instance_file_name in file), None)
    elif dataset == 'itc2007_track3':
        return next((file for file in itc2007_track3_files if instance_file_name in file), None)
    elif dataset == 'itc2019':
        return next((file for file in itc2019_files if instance_file_name + ".xml" in
                     file.split("\\\\")[-1]), None)
    else:
        return None


def parse_problem(dataset, instance_path):
    if dataset == 'itc2007_track2':
        return parse_itc2007_post_enrolment(instance_path)
    elif dataset == 'itc2007_track3':
        return parse_itc2007_curriculum_based(instance_path)
    elif dataset == 'itc2019':
        return parse_xml(instance_path)


def save_solution(dataset, problem, solution_gene, output_folder, raw_room_ids=None, raw_course_ids_for_classes=None,
                  student_classes=None):
    if dataset == 'itc2007_track2':
        output_itc2007_post(problem, solution_gene, os.path.join(output_folder, 'output.sln'))
        return os.path.join(output_folder, 'output.sln')
    elif dataset == 'itc2007_track3':
        output_itc2007_cb(problem, solution_gene, raw_room_ids, raw_course_ids_for_classes,
                          os.path.join(output_folder, 'output.sol'))
        return os.path.join(output_folder, 'output.sol')
    elif dataset == 'itc2019':
        generate_xml(problem, solution_gene, student_classes, os.path.join(output_folder, 'output.xml'))
        return os.path.join(output_folder, 'output.xml')

In [4]:
def get_all_files(path):
    return [os.path.join(dirpath, file) for dirpath, _, filenames in os.walk(path) for file in filenames]


itc2007_track2_files = get_all_files(itc2007_track2_path)
itc2007_track3_files = get_all_files(itc2007_track3_path)
itc2019_files = get_all_files(itc2019_path)


# for each instance, pass through the validator and check if the solution is valid by compiling and running the required cpp file


def validate_itc2007_track3(problem_path, solution_path, results_path):
    import subprocess

    validator_exe = 'D:\\Desktop\\itc validators\\2007cb\\x64\\Release\\2007cb.exe'
    command = [validator_exe, problem_path, solution_path]

    process = subprocess.run(command, capture_output=True)

    output_data = process.stdout

    with open(results_path, 'w') as f:
        f.write(output_data.decode())

    # Check the return code
    if process.returncode != 0:
        print(f"Command failed with exit code {process.returncode}")

    output_string = output_data.decode()

    hard_lines = re.findall(r'\(hard\) : (\d+)', output_string)
    hard_constraint_violations = np.sum([int(line) for line in hard_lines])

    soft_constraint_violations = int(re.search(r'Total Cost = (\d+)', output_string).group(1))

    return hard_constraint_violations, soft_constraint_violations


def validate_itc2019(problem_path, solution_path, results_path):
    # send an api request similar to the following
    #curl \
    #-u email:password \
    #-H "Content-Type:text/xml;charset=UTF-8" \
    #-d @solution.xml \
    #https://www.itc2019.org/itc2019-validator

    with open('itc2019creds.yaml', 'r') as file:
        credentials = yaml.safe_load(file)

    import requests

    with open(solution_path, 'rb') as f:
        solution_data = f.read()

    url = 'https://www.itc2019.org/itc2019-validator'

    headers = {
        'Content-Type': 'text/xml;charset=UTF-8'
    }

    response = requests.post(url, headers=headers, data=solution_data,
                             auth=(credentials['email'], credentials['password']))

    with open(results_path, 'w') as f:
        f.write(response.text)

    data = json.loads(response.text)

    result = data.get('result')
    soft = data.get('totalCost', {}).get('value', -1)

    hard = 0 if result == 'OK' else 999999999

    return hard, soft


In [5]:
def read_stats_file(file_path):
    stats = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.split(": ")
            if key == "Best solution cost":
                stats[key] = tuple(map(int, value.strip()[1:-1].split(", ")))
            elif key == "Execution time":
                stats[key] = float(value.split(" ")[0].strip())
            elif key == "Population init time":
                value = value.strip()[1:-1]
                split_data = value.split("), (")

                tuple_list = []
                for item in split_data:
                    # Remove parentheses from the string
                    item = item.strip("() ")
                    # Split by comma to separate values
                    values = item.split(", ")
                    # Convert the first value to float and the second value to boolean
                    tuple_list.append((float(values[0]), values[1] == "True"))
                stats[key] = tuple_list

    return stats

In [6]:
def plot_history(title,history_df, file_path=None):
    fig, axs = plt.subplots(2)
    
    # Plot hard_cost
    axs[0].plot(history_df['generation'], history_df['hard_cost'])
    axs[0].set_title('Hard Cost')
    axs[0].set(xlabel='Generation', ylabel='Cost')
    
    # Plot soft_cost
    axs[1].plot(history_df['generation'], history_df['soft_cost'])
    axs[1].set_title('Soft Cost')
    axs[1].set(xlabel='Generation', ylabel='Cost')
    
    plt.subplots_adjust(hspace=0.5)
    
    plt.suptitle(title)
    
    if show_graphs:
        plt.show()        
        
    if file_path:
        plt.savefig(file_path)
    
    plt.close()
    
    
def plot_valid_history(title, history_df, file_path=None):
    history_df = history_df[history_df['hard_cost'] == 0]

    fig, ax = plt.subplots()

    ax.plot(history_df['generation'], history_df['soft_cost'])
    ax.set_title('Soft Cost')
    ax.set(xlabel='Generation', ylabel='Cost')

    plt.suptitle(title)

    if show_graphs:
        plt.show()

    if file_path:
        plt.savefig(file_path)

    plt.close()

In [7]:
def read_experiment_stats(experiment_folder, dataset):
    analysis_df = pd.DataFrame(list(os.listdir(experiment_folder)), columns=['folder'])

    analysis_df['folder_path'] = analysis_df.apply(lambda x: os.path.join(experiment_folder, x.folder), axis=1)

    analysis_df['instance_name'] = analysis_df['folder'].apply(lambda x: x.split('_')[0])

    analysis_df['instance_file'] = analysis_df.apply(
        lambda x: get_instance_by_dataset_and_instance_file_name(dataset, x.instance_name), axis=1)

    analysis_df['stats_file'] = analysis_df.apply(
        lambda x: os.path.join(x.folder_path, 'stats.txt'), axis=1)

    analysis_df['stats'] = analysis_df['stats_file'].apply(read_stats_file)

    analysis_df['total_exec_time'] = analysis_df['stats'].apply(lambda x: x['Execution time'])

    analysis_df['population_init_time_arr'] = analysis_df['stats'].apply(lambda x: x['Population init time'])

    analysis_df['population_init_time'] = analysis_df['population_init_time_arr'].apply(
        lambda x: sum([y[0] for y in x]))

    if dataset == 'itc2007_track3':
        analysis_df['problem'], analysis_df['problem_stats'], analysis_df['raw_room_ids'], analysis_df[
            'raw_course_ids_for_classes'] = zip(
            *analysis_df.apply(lambda x: parse_problem(dataset, x['instance_file']), axis=1))
    else:
        analysis_df['problem'], analysis_df['problem_stats'] = zip(
            *analysis_df.apply(lambda x: parse_problem(dataset, x['instance_file']), axis=1))

    analysis_df['checkpoint_file'] = analysis_df.apply(
        lambda x: os.path.join(x.folder_path, 'checkpointsolver.pkl'), axis=1)

    analysis_df['checkpoint'] = analysis_df.apply(lambda x: pd.read_pickle(x.checkpoint_file), axis=1)

    analysis_df['best_solution'] = analysis_df.apply(lambda x: x.checkpoint.get_best_solution()[0], axis=1)

    analysis_df['best_cost'] = analysis_df.apply(lambda x: x.checkpoint.get_best_solution()[1], axis=1)

    if dataset == 'itc2019':
        analysis_df['student_classes'] = analysis_df.apply(lambda x: RandomStudentSectioning(x['problem']).apply(),
                                                           axis=1)

    if dataset == 'itc2007_track2':
        #need to save solution since it wasn't saved when running the experiment
        analysis_df['solution_file_path'] = analysis_df.apply(
            lambda x: save_solution(dataset, x['problem'], x['best_solution'], x['folder_path']), axis=1)
    elif dataset == 'itc2007_track3':
        analysis_df['solution_file_path'] = analysis_df.apply(
            lambda x: os.path.join(x.folder_path, 'output.sol'), axis=1)
    elif dataset == 'itc2019':
        analysis_df['solution_file_path'] = analysis_df.apply(
            lambda x: os.path.join(x.folder_path, 'output.xml'), axis=1)

    analysis_df['cost'] = analysis_df.apply(lambda x:     
        calculate_total_cost_including_student_conflicts(x['problem'],x['best_solution'],x.get('student_classes',{})), axis=1)

    analysis_df['hard_cost'] = analysis_df['cost'].apply(lambda x: x[0])
    analysis_df['soft_cost'] = analysis_df['cost'].apply(lambda x: x[1])

    if dataset == 'itc2007_track3':
        analysis_df['validator_cost'] = analysis_df.apply(
            lambda x: validate_itc2007_track3(x['instance_file'], x['solution_file_path'],
                                              os.path.join(x['folder_path'], 'validation_results.txt')), axis=1)
    elif dataset == 'itc2019':
        analysis_df['validator_cost'] = analysis_df.apply(
            lambda x: validate_itc2019(x['instance_file'], x['solution_file_path'],
                                       os.path.join(x['folder_path'], 'validation_results.txt')), axis=1)


    analysis_df['history_file_path'] = analysis_df.apply(
        lambda x: os.path.join(x.folder_path, 'history.csv'), axis=1)
    
    analysis_df['history'] = analysis_df['history_file_path'].apply(lambda x: pd.read_csv(x))

    return analysis_df


In [8]:
itc2007_track2_ls = read_experiment_stats(
    'D:\Desktop\done tests\select ls full\select itc2007 track 2 with valid dfs construction and 150 generations with ls_itc2007_track2_20240430-001928',
    "itc2007_track2")

itc2007_track3_ls = read_experiment_stats(
    'D:\Desktop\done tests\select ls full\select itc2007 track 3 with valid dfs construction and 150 generations with ls_itc2007_track3_20240430-001937',
    "itc2007_track3")

itc2019_ls = read_experiment_stats(
    'D:\Desktop\done tests\select ls full\select itc2019 with valid dfs construction and 150 generations with ls_itc2019_20240430-001952',
    "itc2019")

In [9]:
itc2007_track2_ils = read_experiment_stats(
    'D:\Desktop\done tests\select ils full\select itc2007t2 with valid dfs construction and 150 gens with ils_itc2007_track2_20240507-123948',
    "itc2007_track2")

itc2007_track3_ils = read_experiment_stats(
    'D:\Desktop\done tests\select ils full\select itc2007 track 3 with valid dfs construction and 150 generations with ils_itc2007_track3_20240505-135802',
    "itc2007_track3")

itc2019_ils = read_experiment_stats(
    'D:\Desktop\done tests\select ils full\select itc2019 with valid dfs construction and 150 generations with ils_itc2019_20240505-135818',
    "itc2019")

In [10]:
itc2007_track2_gsga = read_experiment_stats(
    'D:\\Desktop\\done tests\\select gsga full\\select with valid dfs construction and 150 generations with gsga_itc2007_track2_20240510-235446',
    "itc2007_track2")

itc2007_track3_gsga = read_experiment_stats(
    'D:\\Desktop\\done tests\\select gsga full\\select with valid dfs construction and 150 generations with gsga_itc2007_track3_20240510-235500',
    "itc2007_track3")

itc2019_gsga = read_experiment_stats(
    'D:\\Desktop\\done tests\\select gsga full\\select with valid dfs construction and 150 generations with gsga_itc2019_20240510-235509',
    "itc2019")

In [11]:
itc2007_track2_ls.apply(lambda x: plot_history("Local search progression for "+"ITC2007 track 2 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/progress.png")),axis=1)

itc2007_track3_ls.apply(lambda x: plot_history("Local search progression for "+"ITC2007 track 3 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/progress.png")),axis=1)

itc2019_ls.apply(lambda x: plot_history("Local search progression for "+"ITC2019 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/progress.png")),axis=1)

itc2007_track2_ils.apply(lambda x: plot_history("Iterated local search progression for "+"ITC2007 track 2 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/progress.png")),axis=1)

itc2007_track3_ils.apply(lambda x: plot_history("Iterated local search progression for "+"ITC2007 track 3 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/progress.png")),axis=1)

itc2019_ils.apply(lambda x: plot_history("Iterated local search progression for "+"ITC2019 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/progress.png")),axis=1)

itc2007_track2_gsga.apply(lambda x: plot_history("GSGA progression for "+"ITC2007 track 2 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/progress.png")),axis=1)

itc2007_track3_gsga.apply(lambda x: plot_history("GSGA progression for "+"ITC2007 track 3 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/progress.png")),axis=1)

itc2019_gsga.apply(lambda x: plot_history("GSGA progression for "+"ITC2019 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/progress.png")),axis=1)

0    None
1    None
2    None
3    None
4    None
dtype: object

In [12]:
itc2007_track2_ls.apply(lambda x: plot_valid_history("Local search SC progression for "+"ITC2007 track 2 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/valid progress.png")),axis=1)

itc2007_track3_ls.apply(lambda x: plot_valid_history("Local search SC progression for "+"ITC2007 track 3 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/valid progress.png")),axis=1)

itc2019_ls.apply(lambda x: plot_valid_history("Local search SC progression for "+"ITC2019 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/valid progress.png")),axis=1)

itc2007_track2_ils.apply(lambda x: plot_valid_history("Iterated local search SC progression for "+"ITC2007 track 2 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/valid progress.png")),axis=1)

itc2007_track3_ils.apply(lambda x: plot_valid_history("Iterated local search SC progression for "+"ITC2007 track 3 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/valid progress.png")),axis=1)

itc2019_ils.apply(lambda x: plot_valid_history("Iterated local search SC progression for "+"ITC2019 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/valid progress.png")),axis=1)

itc2007_track2_gsga.apply(lambda x: plot_valid_history("GSGA SC progression for "+"ITC2007 track 2 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/valid progress.png")),axis=1)

itc2007_track3_gsga.apply(lambda x: plot_valid_history("GSGA SC progression for "+"ITC2007 track 3 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/valid progress.png")),axis=1)

itc2019_gsga.apply(lambda x: plot_valid_history("GSGA SC progression for "+"ITC2019 - "+x['instance_name'],x['history'],os.path.join(x['folder_path'],"graphs/valid progress.png")),axis=1)

0    None
1    None
2    None
3    None
4    None
dtype: object

In [13]:
itc2019_ils

,folder,folder_path,instance_name,instance_file,stats_file,stats,total_exec_time,population_init_time_arr,population_init_time,problem,...,best_solution,best_cost,student_classes,solution_file_path,cost,hard_cost,soft_cost,validator_cost,history_file_path,history
0,lums-fal17_rep 1,D:\Desktop\done tests\select ils full\select i...,lums-fal17,D:\Desktop\Datasets\2019\3.late\lums-fal17.xml,D:\Desktop\done tests\select ils full\select i...,"{'Best solution cost': (0, 2388), 'Execution t...",52437.521089,"[(178.48715376853943, False), (284.81183075904...",19533.576239,<models.input.problem.Problem object at 0x0000...,...,"[[15, 45], [14, 13], [4, 16], [3, 30], [17, 42...","(0, 2388)",{},D:\Desktop\done tests\select ils full\select i...,"(0, 2388)",0,2388,"(0, 2388)",D:\Desktop\done tests\select ils full\select i...,Unnamed: 0 generation hard_cost soft_c...
1,lums-spr18_rep 1,D:\Desktop\done tests\select ils full\select i...,lums-spr18,D:\Desktop\Datasets\2019\2.middle\lums-spr18.xml,D:\Desktop\done tests\select ils full\select i...,"{'Best solution cost': (0, 1357), 'Execution t...",33668.178052,"[(159.49844455718994, True), (208.248942852020...",13771.759868,<models.input.problem.Problem object at 0x0000...,...,"[[12, 20], [23, 4], [12, 3], [0, 1], [19, 29],...","(0, 1357)",{},D:\Desktop\done tests\select ils full\select i...,"(0, 1357)",0,1357,"(0, 1357)",D:\Desktop\done tests\select ils full\select i...,Unnamed: 0 generation hard_cost soft_c...
2,muni-fi-fal17_rep 1,D:\Desktop\done tests\select ils full\select i...,muni-fi-fal17,D:\Desktop\Datasets\2019\3.late\muni-fi-fal17.xml,D:\Desktop\done tests\select ils full\select i...,"{'Best solution cost': (0, 5090), 'Execution t...",6523.899548,"[(9.885940790176392, True), (9.815082550048828...",487.153518,<models.input.problem.Problem object at 0x0000...,...,"[[2, 0], [0, 0], [0, 0], [7, 8], [0, 0], [6, 2...","(0, 5090)","{1: [80, 104, 498, 276, 27, 46], 2: [476], 3: ...",D:\Desktop\done tests\select ils full\select i...,"(0, 24315)",0,24315,"(0, 24315)",D:\Desktop\done tests\select ils full\select i...,Unnamed: 0 generation hard_cost soft_c...
3,muni-fi-spr17_rep 1,D:\Desktop\done tests\select ils full\select i...,muni-fi-spr17,D:\Desktop\Datasets\2019\2.middle\muni-fi-spr1...,D:\Desktop\done tests\select ils full\select i...,"{'Best solution cost': (0, 6414), 'Execution t...",8221.952287,"[(25.32409381866455, False), (20.9013924598693...",2964.216249,<models.input.problem.Problem object at 0x0000...,...,"[[4, 0], [6, 0], [5, 1], [2, 3], [1, 0], [1, 1...","(0, 6414)","{1: [20, 29, 42, 476, 58, 77], 2: [101, 102, 1...",D:\Desktop\done tests\select ils full\select i...,"(0, 22509)",0,22509,"(0, 22509)",D:\Desktop\done tests\select ils full\select i...,Unnamed: 0 generation hard_cost soft_c...
4,muni-fsps-spr17_rep 1,D:\Desktop\done tests\select ils full\select i...,muni-fsps-spr17,D:\Desktop\Datasets\2019\1.early\muni-fsps-spr...,D:\Desktop\done tests\select ils full\select i...,"{'Best solution cost': (0, 2036), 'Execution t...",4112.341303,"[(2.7010204792022705, True), (2.47749638557434...",168.971830,<models.input.problem.Problem object at 0x0000...,...,"[[0, 12], [0, 14], [0, 0], [4, 3], [4, 0], [-1...","(0, 2036)","{1: [408, 409, 410, 411, 412, 413, 414, 415, 4...",D:\Desktop\done tests\select ils full\select i...,"(0, 221136)",0,221136,"(0, 221136)",D:\Desktop\done tests\select ils full\select i...,Unnamed: 0 generation hard_cost soft_c...


In [14]:
itc2019_ls[['instance_name', 'hard_cost','soft_cost', 'population_init_time', 'total_exec_time']]

,instance_name,hard_cost,soft_cost,population_init_time,total_exec_time
0,lums-fal17,0,1848,10761.467455,19978.646213
1,lums-spr18,0,853,12550.778537,19272.082253
2,muni-fi-fal17,0,25996,311.074486,2571.123254
3,muni-fi-spr17,0,21127,2017.978540,4278.085902
4,muni-fsps-spr17,0,280889,203.921252,2865.435522


In [15]:
itc2007_track2_ls_copy = itc2007_track2_ls.rename(columns={'hard_cost': 'ls_hard_cost', 'soft_cost': 'ls_soft_cost', 'population_init_time': 'ls_population_init_time', 'total_exec_time': 'ls_total_exec_time'}, copy=True)
itc2007_track3_ls_copy = itc2007_track3_ls.rename(columns={'hard_cost': 'ls_hard_cost', 'soft_cost': 'ls_soft_cost', 'population_init_time': 'ls_population_init_time', 'total_exec_time': 'ls_total_exec_time'}, copy=True)
itc2019_ls_copy = itc2019_ls.rename(columns={'hard_cost': 'ls_hard_cost', 'soft_cost': 'ls_soft_cost', 'population_init_time': 'ls_population_init_time', 'total_exec_time': 'ls_total_exec_time'}, copy=True)

itc2007_track2_ils_copy = itc2007_track2_ils.rename(columns={'hard_cost': 'ils_hard_cost', 'soft_cost': 'ils_soft_cost', 'population_init_time': 'ils_population_init_time', 'total_exec_time': 'ils_total_exec_time'}, copy=True)
itc2007_track3_ils_copy = itc2007_track3_ils.rename(columns={'hard_cost': 'ils_hard_cost', 'soft_cost': 'ils_soft_cost', 'population_init_time': 'ils_population_init_time', 'total_exec_time': 'ils_total_exec_time'}, copy=True)
itc2019_ils_copy = itc2019_ils.rename(columns={'hard_cost': 'ils_hard_cost', 'soft_cost': 'ils_soft_cost', 'population_init_time': 'ils_population_init_time', 'total_exec_time': 'ils_total_exec_time'}, copy=True)

itc2007_track2_gsga_copy = itc2007_track2_gsga.rename(columns={'hard_cost': 'gsga_hard_cost', 'soft_cost': 'gsga_soft_cost', 'population_init_time': 'gsga_population_init_time', 'total_exec_time': 'gsga_total_exec_time'}, copy=True)
itc2007_track3_gsga_copy = itc2007_track3_gsga.rename(columns={'hard_cost': 'gsga_hard_cost', 'soft_cost': 'gsga_soft_cost', 'population_init_time': 'gsga_population_init_time', 'total_exec_time': 'gsga_total_exec_time'}, copy=True)
itc2019_gsga_copy = itc2019_gsga.rename(columns={'hard_cost': 'gsga_hard_cost', 'soft_cost': 'gsga_soft_cost', 'population_init_time': 'gsga_population_init_time', 'total_exec_time': 'gsga_total_exec_time'}, copy=True)

combined_df_itc2007_track2 = pd.merge(itc2007_track2_ls_copy, itc2007_track2_ils_copy, on='instance_name', how='outer')
combined_df_itc2007_track2 = pd.merge(combined_df_itc2007_track2, itc2007_track2_gsga_copy, on='instance_name', how='outer')

combined_df_itc2007_track3 = pd.merge(itc2007_track3_ls_copy, itc2007_track3_ils_copy, on='instance_name', how='outer')
combined_df_itc2007_track3 = pd.merge(combined_df_itc2007_track3, itc2007_track3_gsga_copy, on='instance_name', how='outer')

combined_df_itc2019 = pd.merge(itc2019_ls_copy, itc2019_ils_copy, on='instance_name', how='outer')
combined_df_itc2019 = pd.merge(combined_df_itc2019, itc2019_gsga_copy, on='instance_name', how='outer')


combined_df_itc2007_track2['dataset'] = 'itc2007_track2'
combined_df_itc2007_track3['dataset'] = 'itc2007_track3'
combined_df_itc2019['dataset'] = 'itc2019'

combined_df = pd.concat([combined_df_itc2007_track2, combined_df_itc2007_track3, combined_df_itc2019])

columns_to_keep = ['dataset', 'instance_name', 
                   'ls_population_init_time', 'ls_total_exec_time', 'ls_hard_cost', 'ls_soft_cost',
                   'ils_population_init_time', 'ils_total_exec_time', 'ils_hard_cost', 'ils_soft_cost',
                   'gsga_population_init_time', 'gsga_total_exec_time', 'gsga_hard_cost', 'gsga_soft_cost']

combined_df = combined_df[columns_to_keep]

combined_df.to_excel(os.path.join(assets_folder_path, "results.xlsx"), index=False)


In [16]:
#concat all dfs
concat_df = pd.concat([itc2007_track2_ls, itc2007_track3_ls, itc2019_ls,
                       itc2007_track2_ils, itc2007_track3_ils, itc2019_ils,
                       itc2007_track2_gsga, itc2007_track3_gsga, itc2019_gsga])

concat_df[['instance_name','cost','validator_cost']]

,instance_name,cost,validator_cost
0,comp-2007-2-17,"(0, 466)",NaN
0,comp03,"(0, 545)","(0, 545)"
1,comp12,"(0, 1424)","(0, 1424)"
2,comp14,"(0, 363)","(0, 363)"
3,comp15,"(0, 521)","(0, 521)"
4,comp18,"(0, 235)","(0, 235)"
0,lums-fal17,"(0, 1848)","(0, 1848)"
1,lums-spr18,"(0, 853)","(0, 853)"
2,muni-fi-fal17,"(0, 25996)","(0, 25996)"
3,muni-fi-spr17,"(0, 21127)","(0, 21127)"
